In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

import jax 
import jax.numpy as jnp 

In [45]:
n =  10
max_d  = 9
patch = np.random.normal(size=(10,10,10))

In [46]:
patch.shape

(10, 10, 10)

In [49]:
ids = np.array([[[i**2 + j ** 2 + k**2 for i in range(n)] for j in range(n)] for k in range(n)])
ids = np.where(ids<max_d)
ids

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
        2]),
 array([0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 0, 0, 0, 1, 1,
        2]),
 array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 0, 1, 2, 0, 1,
        0]))

In [8]:
i = np.array([[x for x in range(max_d + 1)] for x in range(max_d + 1)])
dists = i ** 2 + i.transpose() ** 2
dsquare = dists[np.where(dists <= max_d ** 2)]
x = dsquare ** 0.5

dists = np.array([[i**2 + j ** 2  for i in range(max_d+1)] for j in range(max_d+1)])
dsquare = dists[np.where(dists <= max_d ** 2)]
x1 = dsquare ** 0.5
x1 == x

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [69]:
def distmap(max_d, dsquare, dists_shape):
    dist_map = np.zeros(dists_shape)
    for i in range(max_d + 1):
        for j in range(max_d + 1):
            d = i ** 2 + j ** 2
            if d <= max_d ** 2:
                idx, _ = bsearch(dsquare, d - 1e-13, d + 1e-13)
                dist_map[i, j] = idx
    dist_map = dist_map.astype(np.int32) - 1
    return dist_map

def bsearch(x, lower_bound, upper_bound):
    if lower_bound > x[-1] or upper_bound < x[0] or upper_bound < lower_bound:
        return None, None
    lower_idx_a = 1
    lower_idx_b = len(x)
    upper_idx_a = 1
    upper_idx_b = len(x)

    while lower_idx_a + 1 < lower_idx_b or upper_idx_a + 1 < upper_idx_b:
        lw = int(np.floor((lower_idx_a + lower_idx_b) / 2))
        if x[lw - 1] >= lower_bound:
            lower_idx_b = lw
        else:
            lower_idx_a = lw
            if upper_idx_a < lw < upper_idx_b:
                upper_idx_a = lw

        up = int(np.ceil((upper_idx_a + upper_idx_b) / 2))
        if x[up - 1] <= upper_bound:
            upper_idx_a = up
        else:
            upper_idx_b = up
            if lower_idx_a < up < lower_idx_b:
                lower_idx_b = up

    if x[lower_idx_a - 1] >= lower_bound:
        lower_idx = lower_idx_a
    else:
        lower_idx = lower_idx_b
    if x[upper_idx_b - 1] <= upper_bound:
        upper_idx = upper_idx_b
    else:
        upper_idx = upper_idx_a

    if upper_idx < lower_idx:
        return None, None

    return lower_idx, upper_idx

In [87]:
dists = np.array([[[i**2 + j ** 2 + k**2 for i in range(max_d+1)] for j in range(max_d+1)] for k in range(max_d+1)])
dsquare = np.sort(np.unique(dists[dists <= max_d ** 2]))


r = np.zeros(dsquare.shape[0])
c = np.zeros(max_d)

for d in range(max_d):
  c[d] = np.where(dists == d)[0].shape[0]
c = c.astype('int')

mask = np.zeros((n,n))

dist_map = distmap(max_d, dsquare, dists.shape)
valid_dists = np.where(dists != -1) # Relevant indicies

mask = np.zeros((n,n,n))
mask[ids] = 1 
tmp = np.zeros((2*n+1,2*n+1,2*n+1))
tmp[:n,:n,:n] = mask 
tmp_fft = np.fft.fftn(tmp)
c1 = np.fft.ifftn(tmp_fft*np.conj(tmp_fft))
c1 = c1[:max_d+1,:max_d+1,:max_d+1]
c1 = np.round(c.real).astype('int')

c1 == c

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [169]:
patch_fft = jnp.zeros((2*n+1,2*n+1,2*n+1),dtype=jnp.complex64)
patch_fft.at[ids].set(patch[ids])
patch_fft = jnp.fft.fftn(patch_fft)
ACF = jnp.fft.ifftn(patch_fft * jnp.conj(patch_fft))

In [79]:
x = jnp.array([2,3,5])
jnp.searchsorted(x,3)

Array(1, dtype=int32)

In [98]:
dist_map = distmap(max_d, dsquare, dists.shape)
x = jnp.searchsorted(dsquare,dists) - 1 

In [154]:
x = jnp.zeros(dists.shape)
for i in range(max_d + 1):
    for j in range(max_d + 1):
        d = i ** 2 + j ** 2
        if d <= max_d ** 2:
            idx = jnp.searchsorted(dsquare, d)
            x = x.at[i,j].set(idx)

In [166]:
@jax.jit
def turn_ACF_isotropic(ACF,dist_map,valid_dists,num_dists,dists_counts):
    r = jnp.zeros(num_dists)
    corrs_count = jnp.zeros(num_dists)
    for d in zip(*valid_dists):
        id = dist_map[d]
        r[id] += ACF[d]
        corrs_count[id] += dists_counts[d]
    idx = jnp.where(corrs_count != 0)[0]
    r.at[idx].set(r[idx] / corrs_count[idx])
    return r,corrs_count[idx]

In [170]:
r,cnt = turn_ACF_isotropic(ACF, dist_map, valid_dists, dists.shape[0],c)

TypeError: Shapes must be 1D sequences of concrete values of integer type, got (Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>,).
If using `jit`, try using `static_argnums` or applying `jit` to smaller subfunctions.
The error occurred while tracing the function turn_ACF_isotropic at /tmp/ipykernel_2006437/2698531278.py:1 for jit. This concrete value was not available in Python because it depends on the value of the argument num_dists.

In [175]:
r = jnp.zeros(dsquare.shape)
corrs_count = jnp.zeros(dsquare.shape)
for d in zip(*valid_dists):
    id = dist_map[d]
    r.at[id].add(ACF[d])
    corrs_count.at[id].add(c[id])
idx = jnp.where(corrs_count != 0)[0]

/scratch/home/moshemaymon/.conda/envs/moshiko/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=complex64 to dtype=float32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
/scratch/home/moshemaymon/.conda/envs/moshiko/lib/python3.10/site-packages/jax/_src/ops/scatter.py:134: ComplexWarning: Casting complex values to real discards the imaginary part
  return lax_internal._convert_element_type(out, dtype, weak_type)


IndexError: index 14 is out of bounds for axis 0 with size 9